<a href="https://colab.research.google.com/github/kingodjerry/spark_study/blob/main/Spark_%ED%8C%8C%EC%9D%BC%ED%8F%AC%EB%A7%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

PySpark 설치

In [1]:
!pip install PySpark==3.5.3

## Spark Session 만들기

In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
  spark = SparkSession \
          .builder \
          .appName("Spark Writing Demo") \
          .master("local[3]") \
          .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.5.3") \
          .getOrCreate()

In [6]:
!wget  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2024-11-30 13:23:57--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 52.92.190.218, 52.218.168.241, 52.92.242.202, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|52.92.190.218|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv’

appl_stock.csv      100%[===================>] 139.78K   652KB/s    in 0.2s    

2024-11-30 13:23:58 (652 KB/s) - ‘appl_stock.csv’ saved [143130/143130]



In [10]:
df = spark.read \
    .format("csv") \
    .load("appl_stock.csv")

In [11]:
print("Num Partitions before: "+ str(df.rdd.getNumPartitions()))
df.groupBy(spark_partition_id()).count().show() # 같은 id를 묶어서 show

Num Partitions before: 1
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0| 1763|
+--------------------+-----+



In [12]:
df2 = df.repartition(4) # 파티션 수 늘리기
print("Num Partitions after: "+ str(df2.rdd.getNumPartitions()))
df2.groupBy(spark_partition_id()).count().show()

Num Partitions after: 4
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  440|
|                   1|  441|
|                   2|  441|
|                   3|  441|
+--------------------+-----+



In [15]:
df3 = df2.coalesce(2) # 파티션 수 줄이기
print("Num Partitions after: "+ str(df3.rdd.getNumPartitions()))
df3.groupBy(spark_partition_id()).count().show()

Num Partitions after: 2
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  881|
|                   1|  882|
+--------------------+-----+



In [16]:
df.write \
    .format("avro") \
    .mode("overwrite") \
    .option("path", "dataOutput/avro/") \
    .save()

In [17]:
df2.write \
    .format("parquet") \
    .mode("overwrite") \
    .option("path", "dataOutput/parquet/") \
    .save()

In [18]:
df3.write \
    .format("json") \
    .mode("overwrite") \
    .option("path", "dataOutput/json/") \
    .save()

In [19]:
!ls -tl dataOutput/

total 12
drwxr-xr-x 2 root root 4096 Nov 30 13:41 json
drwxr-xr-x 2 root root 4096 Nov 30 13:40 parquet
drwxr-xr-x 2 root root 4096 Nov 30 13:28 avro


In [20]:
!ls -tl dataOutput/parquet/

total 112
-rw-r--r-- 1 root root     0 Nov 30 13:40 _SUCCESS
-rw-r--r-- 1 root root 27847 Nov 30 13:40 part-00003-eb277248-7835-4223-85b3-8ee05428096a-c000.snappy.parquet
-rw-r--r-- 1 root root 27910 Nov 30 13:40 part-00002-eb277248-7835-4223-85b3-8ee05428096a-c000.snappy.parquet
-rw-r--r-- 1 root root 27630 Nov 30 13:40 part-00000-eb277248-7835-4223-85b3-8ee05428096a-c000.snappy.parquet
-rw-r--r-- 1 root root 27652 Nov 30 13:40 part-00001-eb277248-7835-4223-85b3-8ee05428096a-c000.snappy.parquet


In [21]:
!ls -tl dataOutput/avro/

total 88
-rw-r--r-- 1 root root     0 Nov 30 13:28 _SUCCESS
-rw-r--r-- 1 root root 89029 Nov 30 13:28 part-00000-7a5e8f8e-aea6-407f-8984-6a96de9560de-c000.avro
